In [45]:
import numpy as np
import pandas as pd
import re

df = pd.read_csv("data_to_annotate.csv")
df["educational_value_labels"] = df["educational_value_labels"].apply(
    lambda x: re.findall(r"'(.*?)'", x)
)

df_carl = pd.read_json("annotations_carl.json")
df_mikkel = pd.read_json("annotations_mikkel.json")
df_david = pd.read_json("annotations_david.json")


In [46]:
df["all_labels"] = list(zip(
    df["educational_value_labels"],
    df_carl["our_label"].astype(str),
    df_mikkel["our_label"].astype(str),
    df_david["our_label"].astype(str)
))

# Now flatten to a single list per row:
df["all_labels"] = df["all_labels"].apply(lambda x: [x[1], x[2], x[3]] + x[0])

In [47]:
num_encoder = {'None': 0,
               'Minimal': 1,
               'Basic': 2,
               'Good': 3,
               'Excellent': 4}

def to_scores(labels):
    scores = [num_encoder[label] for label in labels]
    return scores

df["all_scores"] = df["all_labels"].apply(to_scores)

In [48]:
def average(all_scores):
    return int(round(sum(all_scores) / len(all_scores)))

df["int_score"] = df["all_scores"].apply(average)

In [49]:
df.to_csv("test_final.csv", index=False)